# Intermediate options

In the previous lesson we talked about the following options:

    display
    maxiter  
    tol 
    tol_step
    tol_improv
    tol_grad
    
These options tend to be the standard in many iterative algorithms and the user may be satisfied with this. In fact, Tensor Fox was constructed in order to be as robust as possible. The several extra options should be used only when the options above are not enough (and when this happens, be sure you are handling a difficult tensor).

The following options will be introduced now:

    tol_mlsvd
    trunc_dims
    initialization
    refine    
    init_damp
    symm    

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import TensorFox as tfx

In [2]:
# Create and print the tensor.
m = 2
T = np.zeros((m, m, m))
s = 0

for k in range(m):
    for i in range(m):
        for j in range(m):
            T[i,j,k] = s
            s += 1

# Truncation

In several applications of linear algebra, often one have to compute the truncated SVD of some matrix. By truncating we can reduce the dimensionality of the problem, which leads to lots of speed up. On the other hand, we lost information after the truncation. Ideally, we want to truncate as much as possible while maintaining the relevant information.

The same can be made for tensors, but in this context we use the [multilinear singular value decomposition](https://epubs.siam.org/doi/abs/10.1137/s0895479896305696) (MLSVD). If $T$ is a $L$-order tensor, then its MLSVD is written as $T = (U_1, \ldots, U_L) \cdot S$, where each $U_l$ is a orthogonal matrix and $S$ is a tensor with the same shape as $T$ (we consider $S$ as the *compressed* version of $T$). The notation used stands for the [multilinear multiplication](https://en.wikipedia.org/wiki/Multilinear_multiplication) brtween the $L$-tuple and the tensor $S$. This tensor $S$ is called the *central tensor* and it is the analogous of $\Sigma$ in the classical SVD of the form $A = U \Sigma V^T$. Note that we've said that $S$ is of the same shape as $T$, and just as in the 2D linear algebra, this is the *full* MLSVD, in contrast to the *reduced* MLSVD. In the same way we can consider the reduced SVD $A = U \Sigma V^T$ where $\Sigma$ is $R \times R$ (and $R$ is the rank of $A$), we can have $S$ of shape $R_1 \times R_2 \times \ldots \times R_L$. The tuple $(R_1, R_2, \ldots, R_L)$ is the *multilinear rank* of $T$. 

The level of trnucation is controlled by the parameter $\verb|tol| \_ \verb|mlsvd|$, which is $10^{-6}$ by default. The program computes the SVD of each unfolding $T_{(\ell)}$. Then it computes the errors 
$$\| T_{(\ell)} - \tilde{T}_{(\ell)} \|^2 / \|T_{(\ell)}\|^2,$$ where $\tilde{T}_{(\ell)}$ is obtained by a truncated SVD of $T_{(\ell)}$. The program increases the rank of the truncations sequentially, until the condition 
$$\| T_{(\ell)} - \tilde{T}_{(\ell)} \|^2 / \|T_{(\ell)}\|^2 < \verb|tol| \_ \verb|mlsvd|$$ 
is satisfied. Two special values are the following.

**1)** $\verb|tol| \_ \verb|mlsvd| = 0$: compress the tensor (that is, compute its MLSVD) but do not truncate the central tensor of the MLSVD.

**2)** $\verb|tol| \_ \verb|mlsvd| = -1$: use the original tensor, so the computation of the MLSVD is not performed.
 
If you are working with tensor with order higher than $3$, then you can pass $\verb|tol| \_ \verb|mlsvd|$ as a list with two values. The first one works for the truncation of the original high order tensor. The second one works for the truncations of the associated tensor train third order CPD's (see next lesson). If you set $\verb|tol| \_ \verb|mlsvd|$ to a single value, the program assumes you want to use this value for both high order and third order tensors. Finally, we want to mention that you can also use the parameter $\verb|trunc| \_ \verb|dims|$ to tell the program the exactly truncation you want to use. Just set this parameter to be a list of the dimensions you want (default is $\verb|trunc| \_ \verb|dims| = 0$, which lets the program to decide the best truncation). If you are work with tensors with order higher than $3$, this parameter refers to the original high order tensor only. 

In the example we are working it is possible to note that the program is unable to truncate. Still, we can force some truncation, say $2 \times 1 \times 1$, and see if we get good precision of this. The precision can already be antecipated just by seeing the relative error of the compression (remember that this requires setting $\verb|display|$ to $3$, which can be costly), that is, the line

    Compression relative error = 0.130828

That line says this is the best precision we can get. This is because this is the error of the truncated $S$ and $T$, and all the iterations to be made will try to obtain a CPD for $S$.

In [3]:
class options:
    display = 1
    trunc_dims = [2,1,1]

R = 3
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compression detected
    Compressing from (2, 2, 2) to (2, 1, 1)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 23
    Relative error = 0.13082808698366935
    Accuracy =  86.91719 %


# Initialization

The iteration process needs a starting point for iterating. This starting point depends on the $\verb|initialization|$ option, and there are three possible choices in this case: $\verb|smart|, \ \verb|smart| \_ \verb|random|, \ \verb|random|$ and $\verb|user|$. Both $\verb|smart|$ and $\verb|smart| \_ \verb|random|$ options generates a CPD of rank $R$ with a strategy relying on the MLSVD. The strategy $\verb|smart|$ maximizes the energy of the initialization wheareas $\verb|smart| \_ \verb|random|$ makes almost the same, but with a chance to take some different entries. These strategies generates starting points with small relative error, so it is already close to the objective tensor. Although this seems to be a good thing, there is also a risk to be close to a local minimum or saddle point, and in this cases these methods will always fail. The $\verb|random|$ is more robust, this option generates a CPD of rank $R$ with entries drawn from the normal distribution. The relative error in this case usually is close to $1$. Finally, there is the $\verb|user|$ option where the user provides a list $[X, Y, Z]$ as starting point. This is a good idea when we already have a close CPD and want to increase its precision.

In [4]:
# Compute the CPD of T with random initialization. 
# Notice we need to set trunc_dims to zero so the program can decide which truncation to use.
options.trunc_dims = 0
options.initialization = 'smart_random'
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    No compression detected
    Working with dimensions (2, 2, 2)
-----------------------------------------------------------------------------------------------
Type of initialization: smart_random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 23
    Relative error = 6.898570500185587e-07
    Accuracy =  99.99993 %


In [5]:
# Compute the CPD of T with user initialization.
X = np.ones((m, R))
Y = np.ones((m, R))
Z = np.ones((m, R))
options.initialization = [X,Y,Z]
factors, info = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    No compression detected
    Working with dimensions (2, 2, 2)
-----------------------------------------------------------------------------------------------
Type of initialization: user
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 7
    Relative error = 0.13079807723237832
    Accuracy =  86.92019 %


# Refinement

As we mentioned before, the user may give an initial CPD as starting point for our iterative algorithm, which may be a good idea when it is desired to increase the precision of the CPD. This process can be done automatically by setting $\verb|refine|$ to True. This option makes the program runs the algorithm two times, where the second run uses the approximated CPD computed in the first run as starting point. However, this second run is made in the original space (the space of the tensor $T$). Ideally, we want to compress and limit ourselves to the compressed version of $T$, but if this is not enough, the $\verb|refine|$ option can squeeze more precision at a cost of working with uncompressed tensors. This options obly work for third order tensors. If you are working with a high order tensor, the program will use this options only for the intermediate third order tensors.

In [6]:
# Compute the CPD of T with refinement.
options.display = 3
options.initialization = 'random'
options.refine = True
factors, output = tfx.cpd(T, R, options)

-----------------------------------------------------------------------------------------------
Computing MLSVD
    No compression detected
    Working with dimensions (2, 2, 2)
    Compression relative error = 5.105819e-16
-----------------------------------------------------------------------------------------------
Type of initialization: random
    Initial guess relative error = 1.142957e+00
-----------------------------------------------------------------------------------------------
Computing CPD
    Iteration | Rel error |  Step size  | Improvement | norm(grad) | Predicted error | # Inner iterations
        1     | 9.97e-01  |  2.18e+00   |  9.97e-01   |  1.59e+01  |    4.09e-01     |        2        
        2     | 1.93e+00  |  5.22e+00   |  9.31e-01   |  5.30e+00  |    2.86e-02     |        3        
        3     | 5.01e-01  |  1.19e+00   |  1.43e+00   |  2.39e+02  |    9.76e-02     |        3        
        4     | 1.25e-01  |  4.48e-01   |  3.76e-01   |  3.91e+01  |    5

# Damping parameter

In the previous section we mentioned that, at each iteration, the program solves a minimization problem. This minimization is of the form

$$\min_x \| Jx - b\|,$$
where $J$ is a tall matrix (the Jacobian matrix of the function of the residuals) and $x, b$ are vectors. There are two problems here: $J$ has too many rows and is not of full rank. In fact, the number of rows of the matrix brings the curse of dimensionality to the problem. One way to avoid this is to consider solving the normal equations

$$J^T J x = J^Tb.$$

Now the matrix has a reasonable size. To solve the problem of lack of full rank we introduce regularization, thus obtaining the new set of equations

$$(J^T J + \mu D) x = J^Tb$$
where $\mu > 0$ is the damping parameter and $D$ is a diagonal matrix. At each iteration the damping parameter is updated following a certain rule, and the user doesn't have influence over this. On the other hand, the user can choose the initial damping parameter factor. More precisely, the first damping parameter is $\mu = \tau \cdot E[T]$, where $\tau$ is the damping parameter factor and $E[T]$ is the mean of the values of $T$ (if there is compression, use $S$ instead of $T$). The default value we use is $\tau = 1$, but the user can change it with the parameter $\verb|init| \_ \verb|damp|$. Experience shows that this value has little influence on the overall process, but sometimes it has a noticeable influence, so be aware of that. Finally, we remark that it is possible to pass the parameter $\verb|init| \_ \verb|damp|$ as a list of values, such that $\verb|init| \_ \verb|damp|[k]$ will be the damping parameter used at $k$-th iteration.   

# Symmetric tensors

If one want to work with symmetric tensors, just set $\verb|symm|$ to True. With this option activated the initialization and all iterations of the dGN function will be done with symmetric tensors. At each iteration the approximated CPD is given by a triplet $X, Y, Z$. The next step is to make the assignements

$$X \leftarrow \frac{X+Y+Z}{3}, \quad Y \leftarrow X, \quad Z \leftarrow X.$$

If the objective tensor is really symmetric, then this procedure converges. Otherwise it can diverge or become unstable.